# $k$-means Clustering

## Intro

#### Import libraries and download dataset

The specific documentation for k-means can be found [here](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

In [ ]:
# Required packages for today
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn import metrics
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# Need scipy for hierarchical clustering visualizations
from scipy.cluster.hierarchy import dendrogram

# Familiar packages for plotting, data manipulation, and numeric functions
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Import a colleague's code for the demo clusters/visualizations
import src.demo_images as demo
import src.k_means_plotter as kmplot
import src.hier_example as hier

# Have plots appear in notebook
%matplotlib inline

# Default plot params
plt.style.use('seaborn')
cmap = 'tab10'

## Clustering!   Finding **GROUPS**

How many groups do you see?

<img src="images/initialscenario.png" width=600>

## Wait - How is clustering different from classification?

>In _classification_ you **know** what groups are in the dataset and the goal is to _**predict**_ class membership accurately.
>
>In _clustering_ you **do not** know which groups are in the dataset and you are trying to _**identify**_ the groups.

### So what do you do with clustering results?

Clustering is often an *informing* step in your analysis. Once clusters are identified, one can:
- Create strategies on how to approach each group differently
- Use cluster membership as an independent variable in a predictive model
- Use the clusters as the _**target label**_ in future classification models. How would you assign new data to the existing clusters?

## Explore the algorithm with an intuitive K means approach

### Observe the following four methods with a sample dataset:

| Method 1 | Method 2 |
| -------- | -------- |
| <img src="images/from-left.gif" width=400> | <img src="images/from-right.gif" width=400> |

| Method 3 | Method 4 |
| -------- | -------- |
| <img src="images/from-top.gif" width=400> | <img src="images/from-bottom.gif" width=400> |

### Method Questions:

- What do they have in common?
- What are the differences between them?
- How many groups are there in the end?
- Do you see any problems with this method?

### K-means algorithm, at its core, in an optimization function

<img src="images/minmaxdata.png" width=400>

### Reassigns groups and adjusts centroids to...

<img src="images/min.png" width=700>

### And to...

<img src="images/max.png" width=700>

**Sci-kit Learn** documentation actually has some pretty good [documentation describing the algorithm](https://scikit-learn.org/stable/modules/clustering.html#k-mean) if you want more detail.

## $k$-Means Plotter - Introducing the Challenges of Clustering

In [ ]:
X, Y = datasets.make_blobs(centers=3)

In [ ]:
plt.scatter(X[:, 0], X[:, 1]);

In [ ]:
np.random.seed(1)
df = kmplot.k_means(X[:, 0], X[:, 1], k=3)

In [ ]:
np.random.seed(42)
df = kmplot.k_means(X[:, 0], X[:, 1], k=3)

In [ ]:
np.random.seed(2)
df = kmplot.k_means(X[:, 0], X[:, 1], k=3)

In [ ]:
np.random.seed(1)
df = kmplot.k_means(X[:, 0], X[:, 1], k=2)

## **Assumptions** and **challenges** of $k$-means

- Demonstrate the ideal $k$-means dataset
- Show three scenarios where $k$-means struggles

#### Ideal $k$-means scenario

In [ ]:
demo.ideal()

#### Meets all assumptions:

- Independent variables
- Balanced cluster sizes
- Clusters have similar density
- Spherical clusters/equal variance of variables


#### Problem Scenario 1 - classes not all round

In [ ]:
demo.messyOne()

#### Problem Scenario 2 - imbalanced class size

In [ ]:
demo.messyTwo()

#### Problem Scenario 3 - class size and density

In [ ]:
demo.messyThree()

#### Solution to challenges:

- Preprocessing: PCA or scaling
- Try a different clustering methods

## Exercise:
### $k$-means on larger dataset - Wine subscription

You want to run a wine subscription service, but you have no idea about wine tasting notes. You are a person of science.
You have a wine dataset of scientific measurements.
If you know a customer likes a certain wine in the dataset, can you recommend other wines to the customer in the same cluster?

#### Questions:
- How many clusters are in the wine dataset?
- What are the characteristics of each clusters?
- What problems do you see potentially in the data?

The dataset is `Wine.csv`

In [ ]:
# Work on problem here: Would scaling make a difference?
wine = pd.read_csv('data/Wine.csv')
wine.drop(columns=['Customer_Segment'], inplace=True) # removing spoilers
wine.head()

In [ ]:
wine.info()

In [ ]:
wine.describe()

In [ ]:
sns.pairplot(wine)

### Start!

In [ ]:
# Let's scale the data first


In [ ]:
# Now - initialize KMeans! How many clusters...?
model = None

In [ ]:
# Let's explore


In [ ]:
# Let's put our labels back into our data
labeled_df = pd.concat([wine, pd.DataFrame(model.labels_,
                        columns=['cluster'])], axis=1)

In [ ]:
labeled_df['cluster'].value_counts()

In [ ]:
# What do these look like?
labeled_df.groupby('cluster').mean()

#### Note! 

You may have different cluster centers - the algorithm is sensitive to starting points.

Even if we set `n_init` to a significant value, it's still a good idea to use `random_state` to ensure repeatable results.

## Choosing the appropriate number for $k$

#### Two metrics we can use: **elbow method** and the **silhouette coefficient**

### Elbow Method

Elbow method uses the sum of squared error (SSE) calculated from each instance of $k$ to find the best value of $k$.

This is sometimes called the "inertia" of the model, and fitted sklearn $k$-means models have an `inertia_` attribute.

Sometimes you will see the SSE divided by the total sum of squares in the dataset (how far is each point from the center of the entire dataset)

Fewer clusters seems better, but inertia will always decrease with _more_ clusters. Hence the idea of looking for an elbow in the plot of inertia vs. $k$.

In [ ]:
model.inertia_

Inertia is the sum of squared distances between points and their cluster center.

In [ ]:
# From a colleague

distortions = []

# Calculate SSE for different K
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=301)
    kmeans.fit(wine_scaled)
    distortions.append(kmeans.inertia_)

# Plot values of SSE
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_title('Elbow curve')
ax.set_xlabel('k')
ax.plot(range(2, 10), distortions)
ax.grid(True)

In [ ]:
# If yellowbrick works for you...
# https://www.scikit-yb.org/en/latest/api/cluster/elbow.html
from yellowbrick.cluster import KElbowVisualizer

# Instantiate the clustering model and visualizer
model = KMeans()

visualizer = KElbowVisualizer(model, k=(2,12), timings=True)

visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()  
plt.show()

### Silhouette Coefficient

![silo](images/silo2.png)

> **a** refers to the average distance between a point and all other points in that cluster.
>
> **b** refers to the average distance between that same point and all other points in clusters to which it does not belong

It is calculated for each point in the dataset, then averaged across all points for one cumulative score.

The Silhouette Coefficient ranges between -1 and 1. The closer to 1, the more clearly defined are the clusters. The closer to -1, the more incorrect assignment.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html



In [ ]:
labels = model.labels_

In [ ]:
metrics.silhouette_score(wine_scaled, labels)

In [ ]:
silhouette_scores = {}

for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=301)
    kmeans.fit(wine_scaled)
    labels = kmeans.labels_
    silhouette_scores[k] = metrics.silhouette_score(X, labels)

In [ ]:
silhouette_scores

In [ ]:
# From a colleague
from matplotlib import cm

km = KMeans(n_clusters=3, random_state=0)
y_km = km.fit_predict(wine_scaled)

cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0]
silhouette_vals = metrics.silhouette_samples(wine_scaled, y_km, metric='euclidean')
y_ax_lower, y_ax_upper = 0, 0
yticks = []

for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[y_km == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(c_silhouette_vals)
    
silhouette_avg = np.mean(silhouette_vals)
plt.axvline(silhouette_avg, color="red", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')

plt.tight_layout()
#plt.savefig('images/11_04.png', dpi=300)
plt.show()

In [ ]:
# If yellowbrick works for you...
# https://www.scikit-yb.org/en/latest/api/cluster/silhouette.html
from yellowbrick.cluster import SilhouetteVisualizer

model = KMeans(3, random_state=42)
visualizer = SilhouetteVisualizer(model, colors='yellowbrick')

visualizer.fit(wine_scaled)    # Fit the data to the visualizer
visualizer.show();             # Finalize and render the figure

### So - How many clusters fit the wine data?

What can you tell me about them?

- 


## Hierarchical Clustering

Hierarchical clustering determines cluster assignments by building a hierarchy. This is implemented by either a bottom-up or a top-down approach:

- **Agglomerative clustering** is the bottom-up approach. It merges the two points that are the most similar until all points have been merged into a single cluster.
- **Divisive clustering** is the top-down approach. It starts with all points as one cluster and splits the least similar clusters at each step until only single data points remain.

These methods produce a tree-based hierarchy of points called a **dendrogram**. Similar to partitional clustering, in hierarchical clustering the number of clusters (k) is often predetermined by the user. Clusters are assigned by cutting the dendrogram at a specified depth that results in k groups of smaller dendrograms.

![dendro](images/dendogram.png)

Unlike many partitional clustering techniques, hierarchical clustering is a deterministic process, meaning cluster assignments won’t change when you run an algorithm twice on the same input data.

The **strengths** of hierarchical clustering methods include:

- They often reveal the finer details about the relationships between data objects
- They provide an interpretable dendrogram

The **weaknesses** of hierarchical clustering methods include:

- They’re computationally expensive with respect to algorithm complexity
- They’re sensitive to noise and outliers

In [ ]:
hier.plot_agglomerative_algorithm()

From sklearn's documentation: https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html

In [ ]:
def plot_dendrogram(model, **kwargs):
    """ Create linkage matrix and then plot the dendrogram"""

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(wine_scaled)
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

#### Thoughts?

- 


---
## Level Up

Using online retail data data from [UCI database](https://archive.ics.uci.edu/ml/datasets/online+retail).

You are looking for patterns so you can get people to buy more, more frequently. 
You might have to create some new variables.

In [ ]:
shopping = pd.read_excel('data/Online Retail.xlsx')

In [ ]:
shopping.tail(20)